One of my mentees did something really cool
https://colab.research.google.com/drive/12MTYkd6eZFbJ7tcuUxhNjStnwLdFunpL#scrollTo=6c219485

Here are some resources to look into
https://www.geeksforgeeks.org/python/pyyaml-fullloader-python/

https://medium.com/python-in-plain-english/using-yaml-file-as-a-small-db-in-python-68ebc61dd490
https://medium.com/@ThinkingLoop/10-pandas-data-cleaning-moves-that-saved-my-career-2fe4e8a50f18



These are the various steps we will take

1. Removing columns that are missing significantly data points  - creating a null report
2. Removing betting odds columns - this is like stealing
3. Confirm whether data columns types are correctly mapped
4. Canonicalize text (and stop death-by-typo)
5. One-hot and categorical the smart way - High-cardinality strings are memory hogs. Use category and selective encoding.
6. Look into Great Expectations data validation


Mindset shifts that changed everything
1. **Be explicit**. If a rule isn’t encoded in code, it’s a rumor.
2. **Prefer immutability**. Reassign results; avoid inplace=True for clarity and chaining.
3. **Choose honesty over prettiness**. A lower KPI with correct math beats a rosy one built on accidental exclusions.
4. **Automate small checks**. Five assertions can protect a million rows.

In [1]:
import pandas as pd
from pathlib import Path

# ===== Defining the data directories ====#
PROJECT_ROOT = Path().absolute().parent.parent
DATA_DIR = PROJECT_ROOT / "datasets" 
COMMON_DATA_DIR = DATA_DIR / "common_data"
INGESTED_DIR = DATA_DIR / "ingested_data"
ingested_csv_file = INGESTED_DIR / "enhanced_dataset.csv"

In [2]:
print(f"📁 Project root: {PROJECT_ROOT}")
print(f"📁 Data directory: {DATA_DIR}")
print(f"📁 Common data directory: {COMMON_DATA_DIR}")
print(f"📁 Ingested data directory: {INGESTED_DIR}")

📁 Project root: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores
📁 Data directory: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores\datasets
📁 Common data directory: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores\datasets\common_data
📁 Ingested data directory: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores\datasets\ingested_data


In [3]:
import pandas as pd
data = pd.read_csv(ingested_csv_file)
data.head()

C:\Users\juliu\AppData\Local\Temp\ipykernel_5988\3313027951.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(ingested_csv_file)


Date   Time    HomeTeam     AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0  16/08/2024  20:00  Man United       Fulham   1.0   0.0   H   0.0   0.0   D   
1  17/08/2024  12:30     Ipswich    Liverpool   0.0   2.0   A   0.0   0.0   D   
2  17/08/2024  15:00     Arsenal       Wolves   2.0   0.0   H   1.0   0.0   H   
3  17/08/2024  15:00     Everton     Brighton   0.0   3.0   A   0.0   1.0   A   
4  17/08/2024  15:00   Newcastle  Southampton   1.0   0.0   H   1.0   0.0   H   

   ... SJA  GBH  GBD  GBA  BSH  BSD  BSA  SBH  SBD  SBA  
0  ... NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
1  ... NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2  ... NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
3  ... NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4  ... NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[5 rows x 166 columns]

In [4]:
data.shape

(6841, 166)

In [5]:
data.columns

Index(['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR',
       ...
       'SJA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA', 'SBH', 'SBD', 'SBA'],
      dtype='object', length=166)

**Initial Deletion of columns - columns that are mostly empty**

In [6]:
print(f"The number of columns are ",data.shape[1], " and the number of the rows are",data.shape[0] )

The number of columns are  166  and the number of the rows are 6841


In [7]:
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total,percent],axis=1, keys=['Total','Percent'])
missing_data.head(20)

Total   Percent
1XBD       6470  0.945768
1XBH       6470  0.945768
1XBA       6470  0.945768
BFE<2.5    6464  0.944891
BFE>2.5    6464  0.944891
BFA        6462  0.944599
BFH        6462  0.944599
BFD        6462  0.944599
BFCA       6461  0.944453
BFCH       6461  0.944453
BFEAHA     6461  0.944453
1XBCA      6461  0.944453
1XBCD      6461  0.944453
BFEAHH     6461  0.944453
BFEH       6461  0.944453
BFEC>2.5   6461  0.944453
BFECAHH    6461  0.944453
BFCD       6461  0.944453
BFECH      6461  0.944453
BFECD      6461  0.944453

In [8]:
new_data = data.drop(missing_data[missing_data['Percent'] > 0.10].index, axis=1)
new_data.isnull().sum()
new_data.head()

Date    HomeTeam     AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0  16/08/2024  Man United       Fulham   1.0   0.0   H   0.0   0.0   D   
1  17/08/2024     Ipswich    Liverpool   0.0   2.0   A   0.0   0.0   D   
2  17/08/2024     Arsenal       Wolves   2.0   0.0   H   1.0   0.0   H   
3  17/08/2024     Everton     Brighton   0.0   3.0   A   0.0   1.0   A   
4  17/08/2024   Newcastle  Southampton   1.0   0.0   H   1.0   0.0   H   

      Referee  ...  WHD    WHA  season_id  competition_name  IWH  IWD  IWA  \
0     R Jones  ...  4.2   5.00  2024-2025               EPL  NaN  NaN  NaN   
1  T Robinson  ...  5.5   1.33  2024-2025               EPL  NaN  NaN  NaN   
2   J Gillett  ...  7.0  17.00  2024-2025               EPL  NaN  NaN  NaN   
3    S Hooper  ...  3.5   2.70  2024-2025               EPL  NaN  NaN  NaN   
4    C Pawson  ...  5.5   8.00  2024-2025               EPL  NaN  NaN  NaN   

   VCH  VCD  VCA  
0  NaN  NaN  NaN  
1  NaN  NaN  NaN  
2  NaN  NaN  NaN  
3  NaN  NaN  NaN  
4  NaN  NaN  NaN  

[5 rows x 39 columns]

In [9]:
new_data.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
       'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA',
       'WHH', 'WHD', 'WHA', 'season_id', 'competition_name', 'IWH', 'IWD',
       'IWA', 'VCH', 'VCD', 'VCA'],
      dtype='object')

In [10]:
print(f"The number of columns are ",new_data.shape[1], " and the number of the rows are",new_data.shape[0] )

The number of columns are  39  and the number of the rows are 6841


In [11]:
total = new_data.isnull().sum().sort_values(ascending=False)
percent = (new_data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
total_rows = new_data.shape[0]


missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

Total   Percent
IWH       565.0  0.082590
IWD       565.0  0.082590
IWA       565.0  0.082590
VCH       381.0  0.055694
VCD       381.0  0.055694
VCA       381.0  0.055694
WHA       199.0  0.029089
WHH       199.0  0.029089
WHD       199.0  0.029089
BWD       145.0  0.021196
BWH       145.0  0.021196
BWA       145.0  0.021196
AwayTeam    1.0  0.000146
Date        1.0  0.000146
HomeTeam    1.0  0.000146
HTHG        1.0  0.000146
FTR         1.0  0.000146
FTAG        1.0  0.000146
FTHG        1.0  0.000146
HTAG        1.0  0.000146

In [12]:
print(f"The number of columns are ",new_data.shape[1], " and the number of the rows are",new_data.shape[0] )

The number of columns are  39  and the number of the rows are 6841


In [13]:
new_data.head(10)

Date       HomeTeam        AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0  16/08/2024     Man United          Fulham   1.0   0.0   H   0.0   0.0   D   
1  17/08/2024        Ipswich       Liverpool   0.0   2.0   A   0.0   0.0   D   
2  17/08/2024        Arsenal          Wolves   2.0   0.0   H   1.0   0.0   H   
3  17/08/2024        Everton        Brighton   0.0   3.0   A   0.0   1.0   A   
4  17/08/2024      Newcastle     Southampton   1.0   0.0   H   1.0   0.0   H   
5  17/08/2024  Nott'm Forest     Bournemouth   1.0   1.0   D   1.0   0.0   H   
6  17/08/2024       West Ham     Aston Villa   1.0   2.0   A   1.0   1.0   D   
7  18/08/2024      Brentford  Crystal Palace   2.0   1.0   H   1.0   0.0   H   
8  18/08/2024        Chelsea        Man City   0.0   2.0   A   0.0   1.0   A   
9  19/08/2024      Leicester       Tottenham   1.0   1.0   D   0.0   1.0   A   

        Referee  ...   WHD    WHA  season_id  competition_name  IWH  IWD  IWA  \
0       R Jones  ...  4.20   5.00  2024-2025               EPL  NaN  NaN  NaN   
1    T Robinson  ...  5.50   1.33  2024-2025               EPL  NaN  NaN  NaN   
2     J Gillett  ...  7.00  17.00  2024-2025               EPL  NaN  NaN  NaN   
3      S Hooper  ...  3.50   2.70  2024-2025               EPL  NaN  NaN  NaN   
4      C Pawson  ...  5.50   8.00  2024-2025               EPL  NaN  NaN  NaN   
5      M Oliver  ...  3.50   2.80  2024-2025               EPL  NaN  NaN  NaN   
6  T Harrington  ...  3.75   2.75  2024-2025               EPL  NaN  NaN  NaN   
7     S Barrott  ...  3.50   2.88  2024-2025               EPL  NaN  NaN  NaN   
8      A Taylor  ...  4.00   1.83  2024-2025               EPL  NaN  NaN  NaN   
9    C Kavanagh  ...  4.33   1.65  2024-2025               EPL  NaN  NaN  NaN   

   VCH  VCD  VCA  
0  NaN  NaN  NaN  
1  NaN  NaN  NaN  
2  NaN  NaN  NaN  
3  NaN  NaN  NaN  
4  NaN  NaN  NaN  
5  NaN  NaN  NaN  
6  NaN  NaN  NaN  
7  NaN  NaN  NaN  
8  NaN  NaN  NaN  
9  NaN  NaN  NaN  

[10 rows x 39 columns]

In [14]:
new_data.sample(10)

Date    HomeTeam     AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
1312  26/12/2021    West Ham  Southampton   2.0   3.0   A   0.0   1.0   A   
2317  02/09/2018     Burnley   Man United   0.0   2.0   A   0.0   2.0   A   
3583  19/12/2015   Newcastle  Aston Villa   1.0   1.0   D   1.0   0.0   H   
1861  08/05/2021   Liverpool  Southampton   2.0   0.0   H   1.0   0.0   H   
5974    06/03/10     Arsenal      Burnley   3.0   1.0   H   1.0   0.0   H   
2668  13/08/2017  Man United     West Ham   4.0   0.0   H   1.0   0.0   H   
6051    24/04/10        Hull   Sunderland   0.0   1.0   A   0.0   1.0   A   
4255    19/10/13   Newcastle    Liverpool   2.0   2.0   D   1.0   1.0   D   
5844    05/12/09  Portsmouth      Burnley   2.0   0.0   H   0.0   0.0   D   
5736    30/08/09  Portsmouth     Man City   0.0   1.0   A   0.0   1.0   A   

         Referee  ...  WHD    WHA  season_id  competition_name   IWH   IWD  \
1312    K Friend  ...  3.7   4.00  2021-2022               EPL  1.83  3.85   
2317      J Moss  ...  3.7   1.65  2018-2019               EPL  5.00  3.75   
3583  M Atkinson  ...  3.2   4.00  2015-2016               EPL  2.00  3.30   
1861    K Friend  ...  5.5  11.00  2020-2021               EPL  1.30  6.00   
5974       C Foy  ...  8.0  21.00  2009-2010               EPL  1.15  6.50   
2668  M Atkinson  ...  5.0  11.00  2017-2018               EPL  1.33  5.30   
6051   L Probert  ...  3.3   3.10  2009-2010               EPL  2.10  3.30   
4255  A Marriner  ...  3.6   1.83  2013-2014               EPL  4.00  3.45   
5844      P Dowd  ...  3.4   4.50  2009-2010               EPL  1.85  3.40   
5736      H Webb  ...  3.8   1.57  2009-2010               EPL  5.30  3.50   

        IWA   VCH   VCD    VCA  
1312   4.10  1.80  3.70   4.20  
2317   1.70  6.25  3.80   1.65  
3583   3.60  2.00  3.50   4.10  
1861   9.50  1.25  6.00  12.00  
5974  15.00  1.12  9.00  23.00  
2668   8.70  1.30  5.50  11.50  
6051   3.10  2.38  3.25   3.00  
4255   1.85  4.40  3.90   1.85  
5844   4.00  1.85  3.50   4.33  
5736   1.65  5.50  3.60   1.62  

[10 rows x 39 columns]

In [15]:
new_data.sample(frac=0.002)

Date     HomeTeam       AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
3078    11/09/16      Swansea        Chelsea   2.0   2.0   D   0.0   1.0   A   
3152    19/11/16   Man United        Arsenal   1.0   1.0   D   0.0   0.0   D   
811   03/09/2022    Brentford          Leeds   5.0   2.0   H   2.0   1.0   H   
193   04/01/2025     Man City       West Ham   4.0   1.0   H   2.0   0.0   H   
1756  17/02/2021      Everton       Man City   1.0   3.0   A   1.0   1.0   D   
4708    02/12/12      Norwich     Sunderland   2.0   1.0   H   2.0   1.0   H   
1228  24/10/2021     West Ham      Tottenham   1.0   0.0   H   0.0   0.0   D   
6115    13/09/08   Portsmouth  Middlesbrough   2.0   1.0   H   0.0   1.0   A   
1972  05/10/2019    Liverpool      Leicester   2.0   1.0   H   1.0   0.0   H   
3886    26/10/14      Burnley        Everton   1.0   3.0   A   1.0   2.0   A   
5725    23/08/09     West Ham      Tottenham   1.0   2.0   A   0.0   0.0   D   
6660    01/01/08      Arsenal       West Ham   2.0   0.0   H   2.0   0.0   H   
3947    07/12/14  Aston Villa      Leicester   2.0   1.0   H   1.0   1.0   D   
1460  21/04/2022      Burnley    Southampton   2.0   0.0   H   2.0   0.0   H   

            Referee  ...  WHD   WHA  season_id  competition_name   IWH   IWD  \
3078     A Marriner  ...  3.6  1.62  2016-2017               EPL  5.10  3.60   
3152     A Marriner  ...  3.1  3.00  2016-2017               EPL  2.60  3.25   
811         R Jones  ...  3.5  3.20  2022-2023               EPL  2.15  3.55   
193     M Salisbury  ...  5.8  8.00  2024-2025               EPL   NaN   NaN   
1756     A Marriner  ...  5.0  1.35  2020-2021               EPL  8.50  5.25   
4708         P Dowd  ...  3.2  3.80  2012-2013               EPL  2.15  3.30   
1228      P Tierney  ...  3.4  2.90  2021-2022               EPL  2.40  3.45   
6115      S Attwell  ...  3.3  3.80  2008-2009               EPL  1.90  3.20   
1972     C Kavanagh  ...  4.6  7.50  2019-2020               EPL  1.47  4.60   
3886     A Marriner  ...  3.6  1.75  2014-2015               EPL  4.20  3.50   
5725  M Clattenburg  ...  3.3  2.30  2009-2010               EPL  3.05  3.20   
6660          C Foy  ...  4.5  8.00  2007-2008               EPL  1.33  4.20   
3947       C Pawson  ...  3.1  3.10  2014-2015               EPL  2.20  3.30   
1460      S Attwell  ...  3.4  2.55  2021-2022               EPL  2.75  3.35   

       IWA   VCH   VCD   VCA  
3078  1.65  5.75  4.00  1.65  
3152  2.75  2.60  3.40  2.90  
811   3.30  2.10  3.50  3.25  
193    NaN   NaN   NaN   NaN  
1756  1.35  9.50  5.25  1.33  
4708  3.20  2.15  3.30  3.90  
1228  2.95  2.30  3.40  2.90  
6115  3.60  2.00  3.45  3.90  
1972  6.50  1.45  4.75  7.00  
3886  1.80  4.60  3.75  1.85  
5725  2.25  3.00  3.25  2.30  
6660  8.00  1.30  5.00  9.00  
3947  3.10  2.45  3.30  3.20  
1460  2.60  2.70  3.25  2.55  

[14 rows x 39 columns]

In [17]:
print(new_data.sample(10))


            Date    HomeTeam        AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
5437    10/11/10   Newcastle       Blackburn   1.0   2.0   A   0.0   1.0   A   
5875    20/12/09    West Ham         Chelsea   1.0   1.0   D   1.0   0.0   H   
5954    17/02/10       Wigan          Bolton   0.0   0.0   D   0.0   0.0   D   
5895    30/12/09  Man United           Wigan   5.0   0.0   H   3.0   0.0   H   
1518  22/05/2022    Man City     Aston Villa   3.0   2.0   H   0.0   1.0   A   
3055    20/08/16   Tottenham  Crystal Palace   1.0   0.0   H   0.0   0.0   D   
849   15/10/2022      Fulham     Bournemouth   2.0   2.0   D   1.0   2.0   A   
690   07/04/2024  Man United       Liverpool   2.0   2.0   D   0.0   1.0   A   
5835    28/11/09    West Ham         Burnley   5.0   3.0   H   3.0   0.0   H   
2805  02/12/2017       Stoke         Swansea   2.0   1.0   H   2.0   1.0   H   

       Referee  ...  WHD    WHA  season_id  competition_name   IWH   IWD  \
5437   M Jones  ...  3.3   4.00  2010-2011 

In [19]:
null_report = (
    new_data.isna()
      .mean()
      .rename("missing_rate")
      .to_frame()
      .assign(n_null=new_data.isna().sum())
      .sort_values("missing_rate", ascending=False)
)
null_report

missing_rate  n_null
IWH                   0.082590     565
IWD                   0.082590     565
IWA                   0.082590     565
VCH                   0.055694     381
VCD                   0.055694     381
VCA                   0.055694     381
WHA                   0.029089     199
WHH                   0.029089     199
WHD                   0.029089     199
BWD                   0.021196     145
BWH                   0.021196     145
BWA                   0.021196     145
AwayTeam              0.000146       1
Date                  0.000146       1
HomeTeam              0.000146       1
HTHG                  0.000146       1
FTR                   0.000146       1
FTAG                  0.000146       1
FTHG                  0.000146       1
HTAG                  0.000146       1
HTR                   0.000146       1
Referee               0.000146       1
HS                    0.000146       1
B365H                 0.000146       1
AR                    0.000146       1
HR                    0.000146       1
AY                    0.000146       1
HY                    0.000146       1
AC                    0.000146       1
HC                    0.000146       1
AF                    0.000146       1
HF                    0.000146       1
AST                   0.000146       1
HST                   0.000146       1
AS                    0.000146       1
B365A                 0.000146       1
B365D                 0.000146       1
season_id             0.000000       0
competition_name      0.000000       0

### Duplicate rows removal


In [18]:
#identify duplicate rows
duplicateRows = new_data[new_data.duplicated()]

#view duplicate rows
duplicateRows

Empty DataFrame
Columns: [Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR, HTHG, HTAG, HTR, Referee, HS, AS, HST, AST, HF, AF, HC, AC, HY, AY, HR, AR, B365H, B365D, B365A, BWH, BWD, BWA, WHH, WHD, WHA, season_id, competition_name, IWH, IWD, IWA, VCH, VCD, VCA]
Index: []

[0 rows x 39 columns]

### Incorrect data types

In [21]:
new_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841 entries, 0 to 6840
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              6840 non-null   object 
 1   HomeTeam          6840 non-null   object 
 2   AwayTeam          6840 non-null   object 
 3   FTHG              6840 non-null   float64
 4   FTAG              6840 non-null   float64
 5   FTR               6840 non-null   object 
 6   HTHG              6840 non-null   float64
 7   HTAG              6840 non-null   float64
 8   HTR               6840 non-null   object 
 9   Referee           6840 non-null   object 
 10  HS                6840 non-null   float64
 11  AS                6840 non-null   float64
 12  HST               6840 non-null   float64
 13  AST               6840 non-null   float64
 14  HF                6840 non-null   float64
 15  AF                6840 non-null   float64
 16  HC                6840 non-null   float64


In [26]:
new_df = new_data.dropna()

print(new_df.to_string())

            Date          HomeTeam          AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR        Referee    HS    AS   HST   AST    HF    AF    HC    AC   HY   AY   HR   AR  B365H  B365D  B365A    BWH    BWD    BWA    WHH    WHD    WHA  season_id competition_name    IWH    IWD    IWA     VCH    VCD     VCA
380   11/08/2023           Burnley          Man City   0.0   3.0   A   0.0   2.0   A       C Pawson   6.0  17.0   1.0   8.0  11.0   8.0   6.0   5.0  0.0  0.0  1.0  0.0   8.00   5.50   1.33   8.75   5.25   1.34   8.00   5.00   1.25  2023-2024              EPL   8.00   5.50   1.35   9.500   5.25   1.330
381   12/08/2023           Arsenal     Nott'm Forest   2.0   1.0   H   2.0   0.0   H       M Oliver  15.0   6.0   7.0   2.0  12.0  12.0   8.0   3.0  2.0  2.0  0.0  0.0   1.18   7.00  15.00   1.17   7.50  15.50   1.12   6.50  12.00  2023-2024              EPL   1.20   7.25  14.00   1.140   7.50  17.000
382   12/08/2023       Bournemouth          West Ham   1.0   1.0   D   0.0   0.0   D       

In [27]:
new_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841 entries, 0 to 6840
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              6840 non-null   object 
 1   HomeTeam          6840 non-null   object 
 2   AwayTeam          6840 non-null   object 
 3   FTHG              6840 non-null   float64
 4   FTAG              6840 non-null   float64
 5   FTR               6840 non-null   object 
 6   HTHG              6840 non-null   float64
 7   HTAG              6840 non-null   float64
 8   HTR               6840 non-null   object 
 9   Referee           6840 non-null   object 
 10  HS                6840 non-null   float64
 11  AS                6840 non-null   float64
 12  HST               6840 non-null   float64
 13  AST               6840 non-null   float64
 14  HF                6840 non-null   float64
 15  AF                6840 non-null   float64
 16  HC                6840 non-null   float64


In [25]:
def inc(x):
    return x + 1


def test_answer():
    assert inc(4) == 5

test_answer()

In [19]:
#Standardize column names: lowercase, snake_case, strip spaces

#Borrowed from - Phoebe


#cohort3_DS_clean = cohort3_DS.copy()
#cohort3_DS_clean.columns = (
   # cohort3_DS_clean.columns
  #  .str.strip()
  #  .str.lower()
   # .str.replace(" ", "_")
    #.str.replace("?", "", regex=False)
#)
#cohort3_DS_clean.head()